In [1]:
from dbfread import DBF
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import datetime



import os
import ee
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
import csv
import datetime
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

import logging
import json

# Configuration du logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

from time import sleep
from pyproj import Transformer


In [2]:
# Lire les couches


profiles = pd.DataFrame(iter(DBF('../data/use/afsp/GIS_Dbf/AfSP012Qry_Profiles.dbf', encoding='latin-1')))

profiles


geo = gpd.read_file('../data/use/afsp/GIS_Shape/AfSP012Qry_GeoPoints.shp')
geo['Longitude'] = geo.geometry.x
geo['Latitude'] = geo.geometry.y



# Fusionner les deux sur 'ProfileID'
merged = pd.merge(profiles, geo[['ProfileID', 'Longitude', 'Latitude']], on='ProfileID', how='left')
profiles_ = merged.dropna(subset=['Longitude', 'Latitude'])

# Afficher les profils géoréférencés
profiles_ = profiles_[['ProfileID', 'Longitude', 'Latitude','T_Year']]
#convert the T_Year to int
profiles_['T_Year'] = pd.to_numeric(profiles_['T_Year'], errors='coerce').astype('Int64')
# afficher toute les date avec le poucentage de chaque année
year_counts = profiles_['T_Year'].value_counts(normalize=True) * 100
# print("\nPourcentage de profils par année :")
print(year_counts)

# Afficher les années avec le pourcentage
year_counts = year_counts.reset_index()
profiles_

#afficher les années  unique
unique_years = profiles_['T_Year'].unique()
print("\nAnnées uniques dans les profils géoréférencés :",np.sort(unique_years))

print("before filtering:", profiles_.shape)
profiles_ = profiles_[profiles_['T_Year'] >= 1982 ]
# profiles_ = profiles_[ profiles_['T_Year'] <= 2013]
print("after filtering year:", profiles_.shape)
available_years = profiles_['T_Year'].unique()

# print("available years:", pd.Series(available_years).sort_values())

# filter longitude and latitude
profiles_ = profiles_[(profiles_['Longitude'] != 0.0) &
                        (profiles_['Latitude'] != 0.0) &
                        (profiles_['Longitude'].notnull()) &
                        (profiles_['Latitude'].notnull())]
print("after filtering coordinates:", profiles_.shape)  

profiles_ = profiles_.drop_duplicates(subset=['Longitude', 'Latitude'], keep='first')
print("after removing duplicates:", profiles_.shape)


profiles_list_afsp =  profiles_[['ProfileID', 'Longitude', 'Latitude','T_Year']].values.tolist()



DBFNotFound: could not find file '../data/use/afsp/GIS_Dbf/AfSP012Qry_Profiles.dbf'

In [4]:
""" ird_data =  pd.read_csv('../data/use/ird/Data_to_Analyze.csv',)


tr = Transformer.from_crs(32628, 4326, always_xy=True)
ird_data[["lon", "lat"]] = ird_data.apply(
    lambda r: tr.transform(r["X_Centroid"], r["Y_Centroid"]),
    axis=1, result_type="expand"
)

ird_data = ird_data.drop(columns=["X_Centroid", "Y_Centroid"])
ird_data = ird_data.rename(columns={"lon": "Longitude", "lat": "Latitude", "Profile_id": "ProfileID"})
# ird_data = ird_data.drop_duplicates(subset=["geometry"], keep="first")
profiles_ird = ird_data[['ProfileID', 'Longitude', 'Latitude']]
profiles_ird = profiles_ird.drop_duplicates(subset=['Longitude', 'Latitude'], keep='first', ignore_index=True)

profiles_ird["T_Year"] = 2016
profiles_ird """

' ird_data =  pd.read_csv(\'../data/use/ird/Data_to_Analyze.csv\',)\n\n\ntr = Transformer.from_crs(32628, 4326, always_xy=True)\nird_data[["lon", "lat"]] = ird_data.apply(\n    lambda r: tr.transform(r["X_Centroid"], r["Y_Centroid"]),\n    axis=1, result_type="expand"\n)\n\nird_data = ird_data.drop(columns=["X_Centroid", "Y_Centroid"])\nird_data = ird_data.rename(columns={"lon": "Longitude", "lat": "Latitude", "Profile_id": "ProfileID"})\n# ird_data = ird_data.drop_duplicates(subset=["geometry"], keep="first")\nprofiles_ird = ird_data[[\'ProfileID\', \'Longitude\', \'Latitude\']]\nprofiles_ird = profiles_ird.drop_duplicates(subset=[\'Longitude\', \'Latitude\'], keep=\'first\', ignore_index=True)\n\nprofiles_ird["T_Year"] = 2016\nprofiles_ird '

In [5]:
ird_data =  pd.read_csv('../data/use/ird/all_profiles.csv',)


tr = Transformer.from_crs(32628, 4326, always_xy=True)
ird_data[["Longitude", "Latitude"]] = ird_data.apply(
    lambda r: tr.transform(r["X_Centroid"], r["Y_Centroid"]),
    axis=1, result_type="expand"
)

ird_data = ird_data.drop(columns=["X_Centroid", "Y_Centroid"])
ird_data = ird_data.rename(columns={"Date": "T_Year"})

ird_data['ProfileID'] = ird_data.index
# ird_data = ird_data.drop_duplicates(subset=["geometry"], keep="first")
profiles_ird = ird_data[['ProfileID', 'Longitude', 'Latitude', 'T_Year']]
profiles_ird = profiles_ird.drop_duplicates(subset=['Longitude', 'Latitude'], keep='first', ignore_index=True)
profiles_ird['ProfileID'] =profiles_ird.index.map(lambda x: f'IRD_{x+1}')  # Assign unique ProfileID based on index
# ProfileID  = Point(longitude, latitude)
map_longitude_latitude_profile_id = profiles_ird.set_index(['Longitude', 'Latitude'])['ProfileID'].to_dict()
# profiles_ird["T_Year"] = 2016
# conver T_Year '01/01/2016' to int(2016)
profiles_ird['T_Year'] = profiles_ird['T_Year'].apply(lambda x: int(x.split("/")[-1]))
profiles_ird
# map_longitude_latitude_profile_id
# save map_longitude_latitude_profile_id  as pd
map_longitude_latitude_profile_id_df = pd.DataFrame(list(map_longitude_latitude_profile_id.items()), columns=['Longitude_Latitude', 'ProfileID'])
map_longitude_latitude_profile_id_df.to_csv('../data/use/ird/map_longitude_latitude_profile_id.csv', index=False)



In [14]:
wosis  = pd.read_csv('../data/use/wosis/WoSIS_2023_December/wosis_202312_profiles.tsv', sep='\t', low_memory=False)
print(wosis.columns)
# wosis_profiles =wosis[['profile_id',  'longitude', 'latitude', 'fao_publication_year']].dropna(axis=0, how='any')
wosis_profiles = wosis[['fao_publication_year']].dropna(axis=0, how='any')
wosis_profiles = wosis_profiles.rename(columns={"profile_id": "ProfileID", "longitude": "Longitude", "latitude": "Latitude", "fao_publication_year": "T_Year"})
# wosis_profiles = wosis_profiles.drop_duplicates(subset=['Longitude', 'Latitude'], keep='first', ignore_index=True)
print("before filtering:", wosis_profiles.shape)
#convert the T_Year to int
wosis_profiles['T_Year'] = wosis_profiles['T_Year'].astype(int)
years = wosis_profiles['T_Year'].unique()
ordered_years = np.sort(years)
print("available years:", ordered_years)
wosis_profiles = wosis_profiles[wosis_profiles['T_Year'] >= 2013 ]
print("after filtering:", wosis_profiles.shape)
wosis_profiles


Index(['profile_id', 'profile_code', 'dataset_code', 'site_id',
       'positional_uncertainty', 'country_name', 'longitude', 'latitude',
       'wrb_reference_soil_group_code', 'wrb_reference_soil_group',
       'wrb_prefix_qualifiers', 'wrb_suffix_qualifiers',
       'wrb_principal_qualifiers', 'wrb_supplementary_qualifiers',
       'wrb_publication_year', 'fao_major_group_code', 'fao_major_group',
       'fao_soil_unit_code', 'fao_soil_unit', 'fao_publication_year',
       'usda_order_name', 'usda_suborder', 'usda_subgroup', 'usda_great_group',
       'usda_publication_year'],
      dtype='object')
before filtering: (38792, 1)
available years: [1974 1997]
after filtering: (0, 1)


,T_Year


In [2]:
zone_villages = pd.read_csv('zones_villages.csv')
print(zone_villages.head())
zone_villages.rename(columns={"id": "ProfileID","lon": "Longitude","lat": "Latitude"}, inplace=True)
zone_villages['T_Year'] = 2020
zone_villages

   id        lon        lat
0   1 -16.549903  14.533038
1   2 -16.549937  14.534548
2   3 -16.549982  14.536586
3   4 -16.550032  14.538797
4   5 -16.550082  14.541036


,ProfileID,Longitude,Latitude,T_Year
0,1,-16.549903,14.533038,2020
1,2,-16.549937,14.534548,2020
2,3,-16.549982,14.536586,2020
3,4,-16.550032,14.538797,2020
4,5,-16.550082,14.541036,2020
...,...,...,...,...
3018,3019,-16.428301,14.485332,2020
3019,3020,-16.428304,14.487493,2020
3020,3021,-16.428478,14.489698,2020
3021,3022,-16.428710,14.491891,2020


In [3]:
# Portée requise par Earth Engine
# SCOPES = ["https://www.googleapis.com/auth/earthengine.readonly"]



# def authenticate_earth_engine():
    
#     creds = None

#     # Vérifie si token.json existe (authentification précédente)
#     if os.path.exists("token.json"):
#         creds = Credentials.from_authorized_user_file("token.json", SCOPES)

#     # Sinon, lance le flow OAuth pour obtenir un token
#     if not creds or not creds.valid:
#         if creds and creds.expired and creds.refresh_token:
#             creds.refresh(Request())
#         else:
#             flow = InstalledAppFlow.from_client_secrets_file(
#                 "credentials.json", SCOPES
                
#             )
#             creds = flow.run_local_server(port=0)

#         # Sauvegarde le token pour les prochaines fois
#         with open("token.json", "w") as token:
#             token.write(creds.to_json())

#     # Initialisation Earth Engine avec les credentials OAuth
#     ee.Initialize(credentials=creds)
#     print(" Earth Engine authentifié avec succès !")

# # Authentification Earth Engine
# authenticate_earth_engine()


# Initialisation Earth Engine
SCOPES = ["https://www.googleapis.com/auth/earthengine.readonly"]

def initialize_earth_engine():
    """Initialise Earth Engine avec les credentials"""
    try:
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
        ee.Initialize(credentials=creds)
        logger.info("Earth Engine initialisé avec succès")
    except Exception as e:
        logger.error(f"Erreur d'initialisation Earth Engine: {str(e)}")
        raise

In [4]:

LANDSAT_COLLECTIONS = {
    # "Landsat-1": {
    #     "years": (1972, 1978),
    #     "available_years": (1972, 1978),
    #     "path": "LANDSAT/LM01/C02/T2",
    #     "bands": ["B4", "B5", "B6", "B7", "QA_PIXEL"]
    # },
    # "Landsat-2": {
    #     "years": (1975, 1981),
    #     "available_years": (1975, 1981),
    #     "path": "LANDSAT/LM02/C02/T1",
    #     "bands": ["B4", "B5", "B6", "B7", "QA_PIXEL"]
    # },
    # "Landsat-3": {
    #     "years": (1978, 1983),
    #     "available_years": (1978, 1983),
    #     "path": "LANDSAT/LM03/C02/T1",
    #     "bands": ["B4", "B5", "B6", "B7", "QA_PIXEL"]
    # },
    # "Landsat-4": {
    #     "years": (1982, 1993),
    #     "available_years": (1982, 1993),
    #     "path": "LANDSAT/LT04/C02/T1_L2",
    #     "bands": ["SR_B1", "SR_B2", "SR_B3", "SR_B4", "SR_B5", "SR_B7", "QA_PIXEL"]
    # },
    # "Landsat-5": {
    #     "years": (1984, 2013),
    #     "available_years": (1984, 2013),
    #     "path": "LANDSAT/LT05/C02/T1_L2",
    #     "bands": ["SR_B1", "SR_B2", "SR_B3", "SR_B4", "SR_B5", "SR_B7", "QA_PIXEL"]
    # },
    # "Landsat-7": {
    #     "years": (1999, 2022),
    #     "available_years": (1999, 2022),
        
    #     "path": "LANDSAT/LE07/C02/T1_L2",
    #     "bands": ["SR_B1", "SR_B2", "SR_B3", "SR_B4", "SR_B5", "SR_B7", "QA_PIXEL"]
    # },
    "Landsat-8": {
        "years": (2013, 2025),  # en fonction de la dernière date disponible
        "available_years": (2013, 2025),  # en fonction de la dernière date disponible
        "path": "LANDSAT/LC08/C02/T1_L2",
        "bands": ["SR_B1", "SR_B2", "SR_B3", "SR_B4", "SR_B5", "SR_B6", "SR_B7", "SR_B10", "SR_B11", "QA_PIXEL"]
    },
    # "Landsat-9": {
    #     "years": (2021, 2025),  # pareil, à adapter
    #     "available_years": (2021, 2025),  # pareil, à adapter
    #     "path": "LANDSAT/LC09/C02/T1_L2",
    #     "bands": ["SR_B1", "SR_B2", "SR_B3", "SR_B4", "SR_B5", "SR_B6", "SR_B7", "SR_B10", "SR_B11", "QA_PIXEL"]
    # }
}

SENTINEL_COLLECTIONS = {
    # "Sentinel-1": {
    #     "years": (2014, 2025),
    #     "available_years": (2014, 2025),
    #     "path": "COPERNICUS/S1_GRD",
    #     "bands": ["VV", "VH"]
    # },
    "Sentinel-2": {
        "years": (2015, 2025),
        "available_years": (2015, 2025),
        "path": "COPERNICUS/S2_SR",
        "bands": ["B1", "B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B9", "B10", "B11", "B12"]
    },
    
 
}

# for sat_name, info in LANDSAT_COLLECTIONS.items():
#     try:
#         collection = ee.ImageCollection(info["path"])
#         # Vérifier si la collection est vide
#         if collection.size().getInfo() == 0:
#             logger.warning(f"Aucune image trouvée pour {sat_name}")
#         else:
#             logger.info(f"Collection {sat_name} initialisée avec succès")
#     except Exception as e:
#         logger.error(f"Erreur lors de l'initialisation de la collection {sat_name}: {str(e)}")


In [5]:


initialize_earth_engine()



def get_appropriate_satellite(year,sat):
    """Trouve le satellite Landsat approprié pour une année donnée"""
    year = int(year)
    
    if year < 1972:
        logger.debug(f"Année {year} avant le début de Landsat, utilisation de 1972 comme proxy")
        year = 1972
    available_sat = []
    for name, info in sat.items():
        start, end = info["years"]
        if start <= year <= end:
            available_sat.append((name, info["path"], info["bands"], year))
 
    return  available_sat[::-1]
      
  




def get_band_info(image, band_name, point):
    try:
        band = image.select(band_name)
        
        # Petite géométrie autour du point
        region = point.buffer(10).bounds()

        stats = band.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=region,
            scale=30,  # résolution Landsat
            bestEffort=True  # évite erreur maxPixels
        ).getInfo()

        return {
            band_name: stats.get(band_name)
        }
    except Exception as e:
        logger.warning(f"Erreur pour la bande {band_name}: {str(e)}")
        return None


def get_sat_data(point, target_year, sat):
    """
    Récupère les données Landsat pour un point et une année cible
    """

    logger.debug(f"Récupération des données Landsat pour {point.getInfo()} pour l'année {target_year} avec le satellite {sat}")
    try:
        # Vérifier les coordonnées valides
        coords = point.coordinates().getInfo()
        if coords == [0, 0] or None in coords:
            logger.warning("Coordonnées invalides - point ignoré")
            error = "get_sat_data 1 : Coordonnées invalides - point ignoré"
            return None, None, None, None ,error

        # Trouver le satellite approprié
        # sat_name, sat_path, target_bands, effective_year = get_appropriate_satellite(target_year)
        available_sat = get_appropriate_satellite(target_year, sat)
        start_date = f"{target_year}-01-01"
        end_date = f"{target_year}-12-31"
        # Période de recherche (±3 ans autour de l'année cible)
        if not available_sat:
            logger.debug(f"Aucun satellite disponible pour l'année {target_year}")
            error = f"get_sat_data 1 : Aucun satellite disponible pour l'année {target_year}"
            return None, None, None, None ,error
        count = 0
        for sat_name, sat_path, target_bands, effective_year in available_sat:
            logger.debug(f"Recherche {sat_name} ({start_date} à {end_date}) pour {coords}")
            
            # Créer la collection d'images
            collection = ee.ImageCollection(sat_path) \
                .filterDate(start_date, end_date) \
                .filterBounds(point) \
                .sort('CLOUD_COVER', True)
            
            # Vérifier si des images sont disponibles
            count = collection.size().getInfo()
            if count > 0:
                logger.debug(f"{count} images trouvées pour {sat_name} ({start_date} à {end_date})")
                # Si des images sont trouvées, on sort de la boucle
                break
            
        if count == 0:
            logger.debug(f"Aucune image trouvée pour {sat_name} {start_date}-{end_date}")
            error = f"get_sat_data 1 : Aucune image trouvée pour  {sat_name} {start_date}-{end_date}"
            return start_date, end_date, sat_name, None ,error

        # Prendre l'image la moins nuageuse
        image = collection.first()
        if not image:
            return start_date, end_date, sat_name, None ,f"Aucune image dans la collection {sat_name}"

        # Vérifie les bandes valides
        available_bands = image.bandNames().getInfo()
        valid_bands = [b for b in target_bands if b in available_bands]

        if not valid_bands:
            return start_date, end_date, sat_name, None, f"Aucune bande valide trouvée dans {available_bands}"

        bands_info = {}
        for band in valid_bands:
            info = get_band_info(image, band, point=point)
            if info:
                bands_info[band] = info[band]

        if not bands_info:
            return start_date, end_date, sat_name, None , "Aucune information de bande récupérée"

        return start_date, end_date, sat_name, bands_info, None
    except Exception as e:
        logger.error(f"Erreur lors de la récupération des données Landsat: {str(e)}")
        return None, None, None, None ,f"get_sat_data 2 : Erreur lors de la récupération des données Landsat: {str(e)}"


def process_row(row, sat=LANDSAT_COLLECTIONS):
    """Traite une ligne du dataframe"""
    try:
        profile_id, lon, lat, year = row
        
        # Créer le point géographique
        point = ee.Geometry.Point(lon, lat)
        
        # Récupérer les données Landsat
        start_date, end_date, sat_name, bands_info ,error = get_sat_data(point, int(year), sat)
        # start_date, end_date, sat_name, bands_info = get_sentinel_data(point, start_date="2015-06-01", end_date="2025-01-01")

        if bands_info is None:
            logger.debug(f"Aucune donnée pour {profile_id}")
            bands_data = {"error": error}
        else:
            bands_data = bands_info

        return {
            'ProfileID': profile_id,
            'Longitude': lon,
            'Latitude': lat,
            'T_Year': year,
            'Start_Date': start_date,
            'End_Date': end_date,
            'Satellite': sat_name,
            
            

        } ,bands_data

    except Exception as e:
        logger.error(f"Erreur pour le profil {row[0]}: {str(e)}")
        return None

def main( output_csv,profiles_list,limit=None):
    """Fonction principale"""
    # Charger les données d'entrée
    # df = pd.read_csv(input_csv)
    # profiles_list = df[['ProfileID', 'Longitude', 'Latitude', 'T_Year']].values.tolist()
    
    # Check if the output file already exists et le creer si nécessaire
    if not os.path.exists(output_csv):
        with open(output_csv, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['ProfileID', 'Longitude', 'Latitude', 'T_Year', 'Start_Date', 'End_Date', 'Satellite'])
        
    already_get_data = pd.read_csv(output_csv)

    # Filtrer les profils déjà traités
    processed_profiles = already_get_data['ProfileID'].tolist()
    profiles_list = [p for p in profiles_list if p[0] not in processed_profiles]
    print(f"Nombre de profils à traiter : {len(profiles_list)}")
    # prendre les 5000 premiers profils pour le test
    if limit:
        profiles_list = profiles_list[:limit]

    results = []
    # Traitement parallèle
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = [executor.submit(process_row, row, sat=LANDSAT_COLLECTIONS) for row in profiles_list]

        for future in tqdm(as_completed(futures), total=len(futures), desc="Traitement des profils"):
            result , bands_data = future.result()
            if result:
                results.append({**result, **bands_data})

    # Sauvegarder les résultats
    output_df = pd.DataFrame(results)
    # data_to_save = already_get_data.append(output_df, ignore_ind"""  """ex=True)
    data_to_save = pd.concat([already_get_data, output_df], ignore_index=True)

    data_to_save.to_csv(output_csv, index=False)
    logger.info(f"Résultats sauvegardés dans {output_csv}")
    





INFO:__main__:Earth Engine initialisé avec succès


In [ ]:
# main("lansat_afsp_results_all.csv", profiles_list_afsp)
# main("lansat_ird_results_sentinel_30.csv", profiles_ird.values.tolist(),10)
# main("wosis_profiles.csv", wosis_profiles.values.tolist(),)
# main("lansat_zones_villages_results_sentinel_30.csv", zone_villages.values.tolist())

Nombre de profils à traiter : 3023


Traitement des profils:   0%|          | 0/3023 [00:00<?, ?it/s]/Users/magayendiaye/projects/soc/env/lib/python3.13/site-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for COPERNICUS/S2_SR! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR

  warnings.warn(warning, category=DeprecationWarning)
Traitement des profils: 100%|██████████| 3023/3023 [2:03:54<00:00,  2.46s/it]  
/var/folders/_t/1b5pb5492xl5wggbfsqmpp780000gn/T/ipykernel_1460/1409649678.py:190: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_to_save = pd.concat([already_get_data, output_df], ignore_index=True)
INFO:__main__:Ré

In [6]:
main("lansat_zones_villages_results_landsat_30.csv", zone_villages.values.tolist())




Nombre de profils à traiter : 3023


Traitement des profils: 100%|██████████| 3023/3023 [24:56<00:00,  2.02it/s]
/var/folders/_t/1b5pb5492xl5wggbfsqmpp780000gn/T/ipykernel_9952/822694871.py:190: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_to_save = pd.concat([already_get_data, output_df], ignore_index=True)
INFO:__main__:Résultats sauvegardés dans lansat_zones_villages_results_landsat_30.csv
